# Network Evaluation Script
This script contains the evaluation procedure on the entire validation set for all trained networks.

# Setup

In [ ]:
!pip install tensorflow==1.15.0
!pip install --upgrade tables

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 40.6MB/s 
     |████████████████████████████████| 512kB 40.5MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=60c1905a2406f1667f3d9fcdfbd4f22f908d081391a443bede0dfb42d6fe1339
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

Package Import and Dependencies

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import time
import shutil as sh
from pathlib import Path
import random
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels
from IPython.display import clear_output

# Authentication for Managing Data
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
rootPath = '/content/drive/My Drive/Data/'

register = np.zeros(1)
while not np.any(register):
    try:
        with open(rootPath + 'Isolation/Testing/FileRegisterClosed.csv','r') as f:
            register = np.genfromtxt(f,delimiter = ",")
    except:
        pass
np.shape(register)


(3565, 2)

Preprocessing Functions


In [ ]:
def decode_TFRecord(exampleProto):
# Read TFRecord file
    # Define features
    featureDescription = {
        'x1': tf.VarLenFeature(dtype=tf.float32),
        'y1': tf.VarLenFeature(dtype=tf.float32),
        'z1': tf.VarLenFeature(dtype=tf.float32),
        'vx1': tf.VarLenFeature(dtype=tf.float32),
        'vy1': tf.VarLenFeature(dtype=tf.float32),
        'vz1':tf.VarLenFeature(dtype=tf.float32),
        'x2': tf.VarLenFeature(dtype=tf.float32),
        'y2': tf.VarLenFeature(dtype=tf.float32),
        'z2': tf.VarLenFeature(dtype=tf.float32),
        'vx2': tf.VarLenFeature(dtype=tf.float32),
        'vy2': tf.VarLenFeature(dtype=tf.float32),
        'vz2':tf.VarLenFeature(dtype=tf.float32),
        'x3': tf.VarLenFeature(dtype=tf.float32),
        'y3': tf.VarLenFeature(dtype=tf.float32),
        'z3': tf.VarLenFeature(dtype=tf.float32),
        'vx3': tf.VarLenFeature(dtype=tf.float32),
        'vy3': tf.VarLenFeature(dtype=tf.float32),
        'vz3': tf.VarLenFeature(dtype=tf.float32),
        'x4': tf.VarLenFeature(dtype=tf.float32),
        'y4': tf.VarLenFeature(dtype=tf.float32),
        'z4': tf.VarLenFeature(dtype=tf.float32),
        'vx4': tf.VarLenFeature(dtype=tf.float32),
        'vy4': tf.VarLenFeature(dtype=tf.float32),
        'vz4': tf.VarLenFeature(dtype=tf.float32),
        'label': tf.VarLenFeature(dtype=tf.int64),
        'time': tf.VarLenFeature(dtype=tf.int64),
        'sats': tf.VarLenFeature(dtype=tf.int64)}

    # Extract features from serialized data
    return  tf.io.parse_single_example(exampleProto, featureDescription)


In [ ]:
def preprocDetInd(dataset,seqLen = 50,sat=0):
    #Set the Satellite viewpoint
    satView = tf.constant(sat,dtype=tf.int64)
    
    sats = tf.sparse.to_dense(dataset['sats'])
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])


    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)

    indices = tf.where((time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    zeros = tf.zeros_like(indices)
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    ds = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels

    dataSlices = ds.map(lambda x: tf.slice(data,x,[seqLen,24]))
    correctLabels = tf.boolean_mask(labels,(time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    correctLabels = tf.reshape((correctLabels > 0),(-1,1))
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    ds3 = tf.data.Dataset.zip((dataSlices,dataLabels))
    return ds3

In [ ]:
def preprocDet(dataset,seqLen = 50):
    #Set the Satellite viewpoint
    
    sats = tf.sparse.to_dense(dataset['sats'])
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])


    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)

    indices = tf.where((time[:-seqLen]<time[seqLen:]))
    zeros = tf.zeros_like(indices)
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    ds = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels

    dataSlices = ds.map(lambda x: tf.slice(data,x,[seqLen,24]))
    
    correctLabels = tf.boolean_mask(labels,(time[:-seqLen]<time[seqLen:]))
    correctLabels = tf.reshape((correctLabels > 0),(-1,1))
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    ds3 = tf.data.Dataset.zip((dataSlices,dataLabels))
    return ds3

In [ ]:
def preprocIso(dataset,seqLen = 50):
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])

    labels = tf.sparse.to_dense(dataset['label'])
    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)

    indices = tf.where(time[:-seqLen]<time[seqLen:])
    zeros = tf.zeros_like(indices)
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    ds = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels

    dataSlices = ds.map(lambda x: tf.slice(data,x,[seqLen,24]))
    correctLabels =  tf.boolean_mask(labels,time[:-seqLen]<time[seqLen:])
    correctLabels = tf.one_hot(correctLabels,36,dtype=tf.int32)
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    ds3 = tf.data.Dataset.zip((dataSlices,dataLabels))
    return ds3

In [ ]:
def preprocIsoInd(dataset,seqLen = 50,sat=0):
    #Set the Satellite viewpoint
    satView = tf.constant(sat,dtype=tf.int64)
    
    sats = tf.sparse.to_dense(dataset['sats'])
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])


    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)

    indices = tf.where((time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    zeros = tf.zeros_like(indices)
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    ds = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels

    dataSlices = ds.map(lambda x: tf.slice(data,x,[seqLen,24]))
    
    correctLabels = tf.boolean_mask(labels,(time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    correctLabels = tf.one_hot(correctLabels,36,dtype=tf.int32)
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    ds3 = tf.data.Dataset.zip((dataSlices,dataLabels))
    return ds3

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues, 
                         annotate=True):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = [classes[x] for x in unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    fig.tight_layout()
    
    # Lines and size
    for pos in np.arange(0.5, len(classes)+0.5):
      ax.axhline(pos, linestyle='--')
      ax.axvline(pos, linestyle='--')
    fig.set_size_inches(15,15)
    
    if annotate:
      # Loop over data dimensions and create text annotations.
      fmt = '.2f' if normalize else 'd'
      thresh = cm.max() / 2.
      for i in range(cm.shape[0]):
          for j in range(cm.shape[1]):
              ax.text(j, i, format(cm[i, j], fmt),
                      ha="center", va="center",
                      color="white" if cm[i, j] > thresh else "black",
                     size=7)
    fig.tight_layout()
    return ax

In [ ]:
def createDetectModel(seq_len=32, batch_size=None, stateful=True, 
              num_units=[32, 32]):
  source = tf.keras.Input(
  name='seed', shape=(seq_len, 24), 
      batch_size=batch_size)
  
  lstm_1 = tf.keras.layers.LSTM(num_units[0], stateful=stateful, return_sequences=True)(source)
  lstm_2 = tf.keras.layers.LSTM(num_units[1], stateful=stateful, return_sequences=False)(lstm_1)
  dense_1 = tf.keras.layers.Dense(64, activation='relu')(lstm_2)
  
  predict = tf.keras.layers.Dense(1, activation='sigmoid')(lstm_2)
  
  return tf.keras.Model(inputs=[source], outputs=[predict])

In [ ]:
def createIsolateModel(seq_len=32, batch_size=None, stateful=True, 
              num_units=[32, 32]):
    source = tf.keras.Input(name='seed', shape=(seq_len, 24), 
                            batch_size=batch_size)
    lstm_1 = tf.keras.layers.LSTM(num_units[0], stateful=stateful, return_sequences=True,dropout=0.1, recurrent_dropout=0.1)(source)
    lstm_2 = tf.keras.layers.LSTM(num_units[1], stateful=stateful, return_sequences=False, dropout=0.1)(lstm_1)
    dense_1 = tf.keras.layers.Dense(100, activation='relu')(lstm_2)
    #last dense layer 50 units for 3 layer network
    predict = tf.keras.layers.Dense(36, activation='softmax')(dense_1)
    
    return tf.keras.Model(inputs=[source], outputs=[predict])

Loading Trained Networks

In [ ]:
detFilesCreated = True
isoFilesCreated = True
naiveIsoTestCreated = True
indWeightIsoTestCreated = False
debug = True
iso = False
det = False 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.enable_eager_execution()

detPath = rootPath + 'Results/Detection/'
isoPath = rootPath + 'Results/Isolation/'

# Parameter Definitions

TFinal = 5602
nSats = 6
# Define model parameters
nTimesteps = 50
if debug:
    batchSize = 4096
else:
    batchSize = 4096


In [ ]:
# Detection Validation set
pathDetTest = rootPath + 'Detection/Testing/'
if debug:
    globDetailed = pathDetTest + 'TestFault_NoFault_*.tfrecord'
else:
    globDetailed = pathDetTest + 'TestFault_NoFault_*.tfrecord'

listDetDetailed = tf.io.gfile.glob(globDetailed)
listDetDetailed = listDetDetailed[:1400]
# Isolation Validation set
pathIsoTest = rootPath + 'Isolation/Testing/'
if debug:
    globDetailed = pathIsoTest +'TestFault_*.tfrecord'
else:
    globDetailed = pathIsoTest + 'TestFault_*.tfrecord'

listIsoDetailed = tf.io.gfile.glob(globDetailed)

Evaluating Detection Models

In [ ]:
# Determine beginning
last = True
det = []
fNames = []
#listDetDetailed = list(np.sort(listDetDetailed))
#with open(detPath + "ModelOutput/"+ "EvalDetectNaiveCorrected.csv",'r') as f:
#    lines = f.readlines()   
#    if not lines[-1][:4] == "Test":
#        
#        fileName = lines[-2][:34]
#        last = False
#    else:
#        fileName = lines[-1][:34]
#    fNames.append(fileName)    
#    ind.append(listDetDetailed.index('/content/drive/My Drive/Data/Detection/Testing/'+fileName))#

#openName = detPath + "ModelOutput/"+ "EvalDetectInd0Corrected.csv"
#print(openName)
#with open(openName,'r') as f:
#    lines = f.readlines()    
#    if not lines[-1][:4] == "Test":
#        fileName = lines[-2][:34]
#        last = False
#    else:
#        fileName = lines[-1][:34]
#    print(fileName)
#    fNames.append(fileName)    
#    det.append(listDetDetailed.index('/content/drive/My Drive/Data/Detection/Testing/'+fileName))
#counter = min(ind)
#print(counter)
#print(listDetDetailed)

In [ ]:
len(listDetDetailed)

1400

In [ ]:

# Load Detection Networks
detectNaive = tf.keras.models.load_model(detPath + 'Corrected/NaiveDetect_weights.02-0.83.hdf5')
detectInd0 = tf.keras.models.load_model(detPath + 'detectInd0C_weights.03-0.86.hdf5')#'detectInd0_weights.06-0.86.hdf5'
detectInd0Stateful = tf.keras.models.load_model(detPath + 'detectInd0_stateful_weights.05-0.83.hdf5')

# Make datasets of individual runs
evalNaiveData = []
evalInd0Data = []
evalIndoStateData = []
detNaiveCorrectedModel = createDetectModel(seq_len=50, batch_size=6*5552, stateful=True, num_units=[128, 128])
detNaiveCorrectedModel.set_weights(detectNaive.get_weights())

detInd0Model = createDetectModel(seq_len=50, batch_size=5552, stateful=True, num_units=[128, 128])
detInd0Model.set_weights(detectInd0Stateful.get_weights())

tS = time.perf_counter()
durAverage = 0

nFiles = len(listDetDetailed)
counter = 0
outputPath = detPath + "ModelOutput/"
if not detFilesCreated :
    with open(outputPath + "EvalDetectNaiveCorrectedNF.csv",mode = 'wb') as evalDN, open(outputPath + "EvalDetectInd0CorrectedNF.csv",mode = 'wb') as evalDI0: #,open(outputPath + "EvalDetectInd0S.csv",mode = 'ab') as evalDI0S:
        for i,fileName in enumerate(listDetDetailed[counter:]):
            fName = fileName[len(pathDetTest):]
            if not fName[:17] == "TestFault_NoFault":
                fParam = float(fileName[-19:-9])
                fSat = int(fileName[-22:-21])
                fThr = int(fileName[-21:-20])
                fType = int(fileName[-24])
            else:
                fParam = 0 
                fSat = -1
                fThr = -1
                fType = 0
            fileListDet = tf.data.TFRecordDataset(fileName)
            decodeDetDataset = fileListDet.map(decode_TFRecord)
            processedDet = decodeDetDataset.flat_map(preprocDet)
            processedDetInd0 = decodeDetDataset.flat_map(lambda x: preprocDetInd(x,50,0))
            
            detDataset = processedDet.batch(batchSize,drop_remainder = False)
            detDataset0 = processedDetInd0.batch(batchSize,drop_remainder = False)

            detSteps = 9 
            #evalDetectNaiveFile = detectNaive.predict(detDataset,steps = detSteps,verbose = 1)
            evalDetectNaiveFile = detNaiveCorrectedModel.predict(processedDet.batch(batch_size=6*5552),steps = 1,verbose = 0)
            evalDetectInd0File = detectInd0.predict(detDataset0,steps = np.ceil(detSteps/6),verbose = 0)
            #evalDetectInd0StatefulFile = detInd0Model.predict(processedDetInd0.batch(batch_size=5552),steps =  1,verbose = 0)

            yTrueDetNaiveStack = [labels.numpy().astype(float) for values,labels in detDataset.take(-1)]   
            yTrueDetNaive = np.concatenate([item for sublist in yTrueDetNaiveStack for item in sublist]).reshape(-1,1)

            yTrueInd0Stack = [labels.numpy().astype(float) for values,labels in detDataset0.take(-1)]  
            yTrueInd0 = np.concatenate([item for sublist in yTrueInd0Stack for item in sublist]).reshape(-1,1)

            update = fName + ',' + str(fSat) + ',' + str(fThr) + ',' + str(fType) + ',' + str(fParam) + '\n'
            evalDN.write(update.encode('utf-8'))
            np.savetxt(evalDN, evalDetectNaiveFile.transpose())
            if not fName[:17] == "TestFault_NoFault":
                np.savetxt(evalDN, yTrueDetNaive.transpose())

            evalDI0.write(update.encode('utf-8'))
            np.savetxt(evalDI0, evalDetectInd0File.transpose())
            if not fName[:17] == "TestFault_NoFault":
                np.savetxt(evalDI0, yTrueInd0.transpose())

            #evalDI0S.write(update.encode('utf-8'))
            #np.savetxt(evalDI0S, evalDetectInd0StatefulFile.transpose())
            #np.savetxt(evalDI0S, yTrueInd0.transpose())
            

            if counter%10 == 0 :
                clear_output()
            counter += 1
            tE = time.perf_counter()
            dur = tE-tS
            if durAverage == 0 :
                durAverage = dur
            else:
                durAverage = 0.9*durAverage +0.1*dur 
            tS = tE
            remainder = (nFiles-counter)*durAverage/60
            print("Average time per file: {} seconds".format(durAverage))
            print("Estimated minutes left: {}".format(remainder))
            print("File counter: {}".format(counter))


Evaluating Isolation Models

In [ ]:
numUnits = [256,356]
if not isoFilesCreated:
    # Load Isolation Networks
    isolateNaive = tf.keras.models.load_model(isoPath + 'NaiveIsolateCorrectedLR0.1/weights.15-0.04.hdf5')#'NaiveIsolateHighIntensity/isolateNaiveHighInt3Layer_weights.10-0.04.hdf5') #'NaiveIsolateCorrectedLR0.01/weights.10-0.03.hdf5')
    isolateInd0 = tf.keras.models.load_model(isoPath + 'isolateInd0_weights.17-0.64.hdf5')
    isolateInd1 = tf.keras.models.load_model(isoPath + 'isolateInd1_weights.15-0.64.hdf5')
    isolateInd2 = tf.keras.models.load_model(isoPath + 'isolateInd2_weights.14-0.64.hdf5')
    isolateInd3 = tf.keras.models.load_model(isoPath + 'isolateInd3_weights.13-0.63.hdf5')
    isolateInd4 = tf.keras.models.load_model(isoPath + 'isolateInd4_weights.15-0.65.hdf5')
    isolateInd5 = tf.keras.models.load_model(isoPath + 'isolateInd5_weights.18-0.64.hdf5')

    naiveweights = isolateNaive.get_weights()
    ind0weights = isolateInd0.get_weights()
    ind1weights = isolateInd1.get_weights()
    ind2weights = isolateInd2.get_weights()
    ind3weights = isolateInd3.get_weights()
    ind4weights = isolateInd4.get_weights()
    ind5weights = isolateInd5.get_weights()


    bSizeEval = 512
    # Define models to cope with changing data shape
    isoNaiveModel = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=[128,128])
    isoInd0Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)
    isoInd1Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)
    isoInd2Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)
    isoInd3Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)
    isoInd4Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)
    isoInd5Model = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=numUnits)

    isoNaiveModel.set_weights(naiveweights)
    isoInd0Model.set_weights(ind0weights)
    isoInd1Model.set_weights(ind1weights)
    isoInd2Model.set_weights(ind2weights)
    isoInd3Model.set_weights(ind3weights)
    isoInd4Model.set_weights(ind4weights)
    isoInd5Model.set_weights(ind5weights)

    #Load Confusion matrices
    cmIso0 = np.load(isoPath + "ModelOutput/EvalIsoInd_C0_CM.npy")
    cmIso1 = np.load(isoPath + "ModelOutput/EvalIsoInd_C1_CM.npy")
    cmIso2 = np.load(isoPath + "ModelOutput/EvalIsoInd_C2_CM.npy")
    cmIso3 = np.load(isoPath + "ModelOutput/EvalIsoInd_C3_CM.npy")
    cmIso4 = np.load(isoPath + "ModelOutput/EvalIsoInd_C4_CM.npy")
    cmIso5 = np.load(isoPath + "ModelOutput/EvalIsoInd_C5_CM.npy")
    #Adjust off-diagonal elements
    cmNorm0 = np.zeros([36,36])
    cmNorm1 = np.zeros([36,36])
    cmNorm2 = np.zeros([36,36])
    cmNorm3 = np.zeros([36,36])
    cmNorm4 = np.zeros([36,36])
    cmNorm5 = np.zeros([36,36])

    for j in range(36):
        colSum0 = np.sum(cmIso0[:,j])
        colSum1 = np.sum(cmIso1[:,j])
        colSum2 = np.sum(cmIso2[:,j])
        colSum3 = np.sum(cmIso3[:,j])
        colSum4 = np.sum(cmIso4[:,j])
        colSum5 = np.sum(cmIso5[:,j])
        for i in range(36):
            if not i == j:
                cmNorm0[i,j] = cmIso0[i,j] * -1/colSum0
                cmNorm1[i,j] = cmIso1[i,j] * -1/colSum1
                cmNorm2[i,j] = cmIso2[i,j] * -1/colSum2
                cmNorm3[i,j] = cmIso3[i,j] * -1/colSum3
                cmNorm4[i,j] = cmIso4[i,j] * -1/colSum4
                cmNorm5[i,j] = cmIso5[i,j] * -1/colSum5
            else:
                cmNorm0[i,j] = cmIso0[i,j] / colSum0
                cmNorm1[i,j] = cmIso1[i,j] / colSum1
                cmNorm2[i,j] = cmIso2[i,j] / colSum2
                cmNorm3[i,j] = cmIso3[i,j] / colSum3
                cmNorm4[i,j] = cmIso4[i,j] / colSum4
                cmNorm5[i,j] = cmIso5[i,j] / colSum5


In [ ]:
# Determine beginning
last = True
ind = []
fNames = []
#listIsoDetailed = list(np.sort(listIsoDetailed))
#with open(isoPath + "ModelOutput/"+ "EvalIsoNaive_Net2.csv",'r') as f:
#    lines = f.readlines()    
#    if not lines[-1][:4] == "Test":
#        fileName = lines[-2][:34]
#        last = False
#    else:
#        fileName = lines[-1][:34]
#    fNames.append(fileName)    
#    ind.append(listIsoDetailed.index('/content/drive/My Drive/Data/Isolation/Testing/'+fileName))

#for i in range(6):
#    openName = isoPath + "ModelOutput/"+ "EvalIsoInd" +str(i)+".csv"
#    print(openName)
#    with open(openName,'r') as f:
#        lines = f.readlines()    
#        if not lines[-1][:4] == "Test":
#            fileName = lines[-2][:34]
#            last = False
#        else:
#            fileName = lines[-1][:34]
#        print(fileName)
#        fNames.append(fileName)    
#        ind.append(listIsoDetailed.index('/content/drive/My Drive/Data/Isolation/Testing/'+fileName))
with open(isoPath + "ModelOutput/"+ "EvalIsoIndCombined_C.csv",'r') as f:
    lines = f.readlines()    
    if not lines[-1][:4] == "Test":
        fileName = lines[-2][:34]
        last = False
    else:
        fileName = lines[-1][:34]
    fNames.append(fileName)    
    ind.append(listIsoDetailed.index('/content/drive/My Drive/Data/Isolation/Testing/'+fileName))

counter = min(ind)+1
print(counter)
print(listIsoDetailed)

7128
['/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_31_0.76247000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_56_0.76247000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_55_0.76300000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_56_0.76328000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_15_0.76332000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_24_0.76347000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_61_0.76379000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_65_0.76388000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_22_0.76436000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_32_0.76449000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1_26_0.76453000.tfrecord', '/content/drive/My Drive/Data/Isolation/Testing/TestFault_1

In [ ]:

durAverage = 0
nFiles = len(listIsoDetailed)
outputPath = isoPath + "ModelOutput/"
dataSize = 5602*6#len(yTrueIsoNaive)
dataSizeInd = dataSize//6

tS = time.perf_counter()
# ), open(outputPath + "EvalIsoInd0.csv",mode = 'ab') as (evalII0
#    ), open(outputPath + "EvalIsoInd1.csv",mode = 'ab') as (evalII1
#    ), open(outputPath + "EvalIsoInd2.csv",mode = 'ab') as (evalII2
#    ), open(outputPath + "EvalIsoInd3.csv",mode = 'ab') as (evalII3 
#    ), open(outputPath + "EvalIsoInd4.csv",mode = 'ab') as (evalII4
#    ), open(outputPath + "EvalIsoInd5.csv",mode = 'ab') as (evalII5)  :
#    open(outputPath + "EvalIsoIndCombined.csv",mode = 'wb') as (evalIIC):

if not isoFilesCreated:
    with open(outputPath + "EvalIsoIndCombined_C.csv",mode = 'ab') as (evalIIC):
        for i,fileName in enumerate(listIsoDetailed[counter:]):
            fName = fileName[-34:]
            fParam = float(fileName[-19:-9])
            fSat = int(fileName[-22:-21])
            fThr = int(fileName[-21:-20])
            fType = int(fileName[-24])
            fileListIso = tf.data.TFRecordDataset(fileName)
            decodeIsoDataset = fileListIso.map(decode_TFRecord)
            #processedIso = decodeIsoDataset.flat_map(preprocIso)

            processedIsoInd0 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,0))
            processedIsoInd1 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,1))
            processedIsoInd2 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,2))
            processedIsoInd3 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,3))
            processedIsoInd4 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,4))
            processedIsoInd5 = decodeIsoDataset.flat_map(lambda x: preprocIsoInd(x,50,5))
            #yTrueIsoNaive = [labels for _,labels in processedIso.take(-1)]


            #isoDataset  = processedIso.batch(bSizeEval,drop_remainder = True)
            isoDataset0 = processedIsoInd0.batch(bSizeEval,drop_remainder = True)
            isoDataset1 = processedIsoInd1.batch(bSizeEval,drop_remainder = True)
            isoDataset2 = processedIsoInd2.batch(bSizeEval,drop_remainder = True)
            isoDataset3 = processedIsoInd3.batch(bSizeEval,drop_remainder = True)
            isoDataset4 = processedIsoInd4.batch(bSizeEval,drop_remainder = True)
            isoDataset5 = processedIsoInd5.batch(bSizeEval,drop_remainder = True)
            #evalIsolateNaiveFile = np.argmax(isoNaiveModel.predict(isoDataset,steps = np.floor(dataSize/bSizeEval),verbose = 0),axis = 1)
            
            evalIsolateInd0 = isoInd0Model.predict(isoDataset0,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateInd1 = isoInd1Model.predict(isoDataset1,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateInd2 = isoInd2Model.predict(isoDataset2,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateInd3 = isoInd3Model.predict(isoDataset3,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateInd4 = isoInd4Model.predict(isoDataset4,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateInd5 = isoInd5Model.predict(isoDataset5,steps = np.floor(dataSizeInd/bSizeEval),verbose = 0)
            evalIsolateCombined = np.transpose(np.matmul(cmNorm0, np.transpose(evalIsolateInd0)) + 
                                               np.matmul(cmNorm1, np.transpose(evalIsolateInd1)) + 
                                               np.matmul(cmNorm2, np.transpose(evalIsolateInd2)) + 
                                               np.matmul(cmNorm3, np.transpose(evalIsolateInd3)) + 
                                               np.matmul(cmNorm4, np.transpose(evalIsolateInd4)) + 
                                               np.matmul(cmNorm5, np.transpose(evalIsolateInd5)))
            #evalIsolateInd0File = np.argmax(evalIsolateInd0, axis = 1)
            #evalIsolateInd1File = np.argmax(evalIsolateInd1, axis = 1)
            #evalIsolateInd2File = np.argmax(evalIsolateInd2, axis = 1)
            #evalIsolateInd3File = np.argmax(evalIsolateInd3, axis = 1)
            #evalIsolateInd4File = np.argmax(evalIsolateInd4, axis = 1)
            #evalIsolateInd5File = np.argmax(evalIsolateInd5, axis = 1)
            evalIsolateCombinedFile = np.argmax(evalIsolateCombined, axis = 1)
            #yTrueIsoNaiveStack = [labels.numpy().astype(float) for values,labels in isoDataset.take(-1)]   
            #yTrueIsoNaive = np.concatenate([item for sublist in yTrueIsoNaiveStack for item in sublist]).reshape(-1,1)

            #yTrueInd0Stack = [labels.numpy().astype(float) for values,labels in isoDataset0.take(-1)]  
            #yTrueInd0 = np.concatenate([item for sublist in yTrueInd0Stack for item in sublist]).reshape(-1,1)

            #yTrueInd1Stack = [labels.numpy().astype(float) for values,labels in isoDataset1.take(-1)]  
            #yTrueInd1 = np.concatenate([item for sublist in yTrueInd1Stack for item in sublist]).reshape(-1,1)

            #yTrueInd2Stack = [labels.numpy().astype(float) for values,labels in isoDataset2.take(-1)]  
            #yTrueInd2 = np.concatenate([item for sublist in yTrueInd2Stack for item in sublist]).reshape(-1,1)

            #yTrueInd3Stack = [labels.numpy().astype(float) for values,labels in isoDataset3.take(-1)]  
            #yTrueInd3 = np.concatenate([item for sublist in yTrueInd3Stack for item in sublist]).reshape(-1,1)

            #yTrueInd4Stack = [labels.numpy().astype(float) for values,labels in isoDataset4.take(-1)]  
            #yTrueInd4 = np.concatenate([item for sublist in yTrueInd4Stack for item in sublist]).reshape(-1,1)

            #yTrueInd5Stack = [labels.numpy().astype(float) for values,labels in isoDataset5.take(-1)]  
            #yTrueInd5 = np.concatenate([item for sublist in yTrueInd5Stack for item in sublist]).reshape(-1,1)

            update = fName + ',' + str(fSat) + ',' + str(fThr) + ',' + str(fType) + ',' + str(fParam) + '\n'
            #evalIN.write(update.encode('utf-8'))
            #np.savetxt(evalIN, evalIsolateNaiveFile.reshape(-1,1).transpose(),fmt = '%i')

            #evalII0.write(update.encode('utf-8'))
            #np.savetxt(evalII0, evalIsolateInd0File.reshape(-1,1).transpose(),fmt = '%i')

            #evalII1.write(update.encode('utf-8'))
            #np.savetxt(evalII1, evalIsolateInd1File.reshape(-1,1).transpose(),fmt = '%i')

            #evalII2.write(update.encode('utf-8'))
            #np.savetxt(evalII2, evalIsolateInd2File.reshape(-1,1).transpose(),fmt = '%i')

            #evalII3.write(update.encode('utf-8'))
            #np.savetxt(evalII3, evalIsolateInd3File.reshape(-1,1).transpose(),fmt = '%i')

            #evalII4.write(update.encode('utf-8'))
            #np.savetxt(evalII4, evalIsolateInd4File.reshape(-1,1).transpose(),fmt = '%i')

            #evalII5.write(update.encode('utf-8'))
            #np.savetxt(evalII5, evalIsolateInd5File.reshape(-1,1).transpose(),fmt = '%i')

            evalIIC.write(update.encode('utf-8'))
            np.savetxt(evalIIC, evalIsolateCombinedFile.reshape(-1,1).transpose(),fmt = '%i')
            

            #counterFile.write((fName + ','+str(counter) + '\n').encode('utf-8'))

            counter += 1
            if counter%10 == 0 :
                clear_output()
            tE = time.perf_counter()
            dur = tE-tS
            if durAverage == 0 :
                durAverage = dur
            else:
                durAverage = 0.9*durAverage +0.1*dur 
            tS = tE
            remainder = (nFiles-counter)*durAverage/60
            print("Average time per file: {0:.1f} seconds".format(durAverage))
            print("Estimated minutes left: {0:.1f}".format(remainder) )
            print("File counter: {}".format(counter))



In [ ]:
counter


7128

In [ ]:
naiveTestList = listIsoDetailed
#Get naiveTestList in random, but consistent order
seed = 6
np.random.seed(seed)
naiveTestList = np.sort(naiveTestList)
np.random.shuffle(naiveTestList)
naiveTestList = naiveTestList[:len(naiveTestList)//20]

In [ ]:
i = 0
strAdd = '0' if i<9 else ''
weightsFileName = isoPath + "NaiveIsolateCorrectedLR0.1/weights." +strAdd+ str(i+1)+ "*"
weightsGlob = tf.io.gfile.glob(weightsFileName)[0]
weightsGlob

'/content/drive/My Drive/Data/Results/Isolation/NaiveIsolateCorrectedLR0.1/weights.01-0.04.hdf5'

In [ ]:
numUnits = [256,356]
counter = 0 
if not naiveIsoTestCreated:
    for i in range(10,11):
        # Load Isolation Networks
        strAdd = '0' if i<9 else ''
        weightsFileName = isoPath + "NaiveIsolateCorrectedLR0.1/weights." +strAdd+ str(i+1)+ "*"
        weightsGlob = tf.io.gfile.glob(weightsFileName)[0]
        isolateNaive = tf.keras.models.load_model(weightsGlob)#'NaiveIsolateHighIntensity/isolateNaiveHighInt3Layer_weights.10-0.04.hdf5') #'NaiveIsolateCorrectedLR0.01/weights.10-0.03.hdf5')

        naiveweights = isolateNaive.get_weights()

        bSizeEval = 512
        # Define models to cope with changing data shape
        isoNaiveModel = createIsolateModel(seq_len=50, batch_size=bSizeEval, stateful=True, num_units=[128,128])

        isoNaiveModel.set_weights(naiveweights)

        durAverage = 0
        nFiles = len(naiveTestList)
        outputPath = isoPath + "ModelOutput/NaiveTest/"
        dataSize = 5602*6#len(yTrueIsoNaive)
        dataSizeInd = dataSize//6

        tS = time.perf_counter()
        # ), open(outputPath + "EvalIsoInd0.csv",mode = 'ab') as (evalII0
        #    ), open(outputPath + "EvalIsoInd1.csv",mode = 'ab') as (evalII1
        #    ), open(outputPath + "EvalIsoInd2.csv",mode = 'ab') as (evalII2
        #    ), open(outputPath + "EvalIsoInd3.csv",mode = 'ab') as (evalII3 
        #    ), open(outputPath + "EvalIsoInd4.csv",mode = 'ab') as (evalII4
        #    ), open(outputPath + "EvalIsoInd5.csv",mode = 'ab') as (evalII5)  :
        #    open(outputPath + "EvalIsoIndCombined.csv",mode = 'wb') as (evalIIC):

        with open(outputPath + "NaiveTest_Epoch"+str(i+1)+".csv",mode = 'wb') as (evalIN):
            for i,fileName in enumerate(naiveTestList):
                fName = fileName[-34:]
                fParam = float(fileName[-19:-9])
                fSat = int(fileName[-22:-21])
                fThr = int(fileName[-21:-20])
                fType = int(fileName[-24])
                fileListIso = tf.data.TFRecordDataset(fileName)
                decodeIsoDataset = fileListIso.map(decode_TFRecord)
                processedIso = decodeIsoDataset.flat_map(preprocIso)

                isoDataset  = processedIso.batch(bSizeEval,drop_remainder = True)
                
                evalIsolateNaiveFile = np.argmax(isoNaiveModel.predict(isoDataset,steps = np.floor(dataSize/bSizeEval),verbose = 0),axis = 1)
                
               
                update = fName + ',' + str(fSat) + ',' + str(fThr) + ',' + str(fType) + ',' + str(fParam) + '\n'
                evalIN.write(update.encode('utf-8'))
                np.savetxt(evalIN, evalIsolateNaiveFile.reshape(-1,1).transpose(),fmt = '%i')

                
                counter += 1
                if counter%10 == 0 :
                    clear_output()
                tE = time.perf_counter()
                dur = tE-tS
                if durAverage == 0 :
                    durAverage = dur
                else:
                    durAverage = 0.9*durAverage +0.1*dur 
                tS = tE
                remainder = ((20-11)*nFiles-counter)*durAverage/60
                print("Average time per file: {0:.1f} seconds".format(durAverage))
                print("Estimated minutes left: {0:.1f}".format(remainder) )
                print("File counter: {}".format(counter))



In [ ]:
pathTrain = rootPath + 'Isolation/Training/'
listdirTrain = []
batchSize = 4096

globTrain = pathTrain + 'TrainCorrected_[0-9].tfrecord'
glob2 = pathTrain + 'TrainCorrected_[1-5][0-9].tfrecord'
glob3 = pathTrain + 'TrainCorrected_[6][0-3].tfrecord'
listdirTrain = tf.io.gfile.glob(globTrain) + tf.io.gfile.glob(glob2) + tf.io.gfile.glob(glob3)

fileListDatasetTrain = tf.data.TFRecordDataset(listdirTrain)
decodedDataset = fileListDatasetTrain.map(decode_TFRecord)
processedDataset = decodedDataset.flat_map(lambda x: preprocIsoInd(x,50,0))
trainDataset = processedDataset
trainDataset = trainDataset.batch(batchSize,drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)

# Validation set
pathTest = rootPath + 'Isolation/Testing/'
globTest = pathTest + 'TestCorrected_[0-9].tfrecord'
listdirTest = tf.io.gfile.glob(globTest)
nFilesTest = len(listdirTest)

fileListDatasetTest = tf.data.TFRecordDataset(listdirTest)
decodeValDataset = fileListDatasetTest.map(decode_TFRecord)
processedVal = decodeValDataset.flat_map(lambda x: preprocIsoInd(x,50,0))
validationDataset = processedVal.batch(batchSize,drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)
# Determine steps per epoch
nFilesTrain = len(listdirTrain)
nFilesTest = len(listdirTest)
trainSteps = int(nFilesTrain*100*(TFinal-nTimesteps)/batchSize)
testSteps = int(nFilesTest*100*(TFinal-nTimesteps)/batchSize)
print("Training Files: {}".format(nFilesTrain))
print("Testing Files: {}".format(nFilesTest))


Training Files: 64
Testing Files: 10


In [ ]:
folder = "IndIsolateRegularData/"
fileN = "isolateInd0_weights*"

# Regular Dataset over epochs
weightsFileName = isoPath + folder+fileN 
weightsGlob = tf.io.gfile.glob(weightsFileName)
weightsGlob = np.sort(weightsGlob)

In [ ]:
weightsGlob

array(['/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.01-0.46.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.02-0.54.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.03-0.57.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.04-0.58.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.05-0.59.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.06-0.61.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.07-0.61.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.08-0.61.hdf5',
       '/content/drive/My Drive/Data/Results/Isolation/IndIsolateRegularData/isolateInd0_weights.09-0.63

In [ ]:
numUnits = [256,356]
dataSetSize = "Double Size"
counter = 0 
folder = "IndIsolate_DoubleData/"
fileN = "isolateInd0_DoubleData_weights*"
if not indWeightIsoTestCreated:
    # Regular Dataset over epochs
    weightsFileName = isoPath + folder+fileN 
    weightsGlob = tf.io.gfile.glob(weightsFileName)
    weightsGlob = np.sort(weightsGlob)
    for weightsName in weightsGlob[18:]:
        epoch = int(weightsName[len(isoPath + folder+fileN ):-10])
        # Load Isolation Networks
        isolateNaive = tf.keras.models.load_model(weightsName)#'NaiveIsolateHighIntensity/isolateNaiveHighInt3Layer_weights.10-0.04.hdf5') #'NaiveIsolateCorrectedLR0.01/weights.10-0.03.hdf5')

        indweights = isolateNaive.get_weights()

        # Define models to cope with changing data shape
        isoInd0Model = createIsolateModel(seq_len=50, batch_size=batchSize, stateful=True, num_units=numUnits)
        isoInd0Model.set_weights(indweights)
        adams = tf.keras.optimizers.Adam(learning_rate=0.001)

        isoInd0Model.compile(optimizer=adams,
                        loss=tf.keras.losses.CategoricalCrossentropy(),
                        metrics=['categorical_accuracy']) # Compile with adam optimizer
        durAverage = 0
        nFiles = len(naiveTestList)
        outputPath = isoPath + "ModelOutput/IndTest/"

        print("Individual Model {0} in Epoch {1} on Training Set".format(dataSetSize,str(epoch)))
        isoInd0Model.evaluate(trainDataset,steps = trainSteps)
        #print("Individual Model {0} in Epoch {1} on Validation Set".format(dataSetSize,str(epoch)))
        #isoInd0Model.evaluate(validationDataset,steps = testSteps)
 

Individual Model Double Size in Epoch 13 on Training Set
8675/8675 [==============================] - 2318s 267ms/step - loss: 1.2254 - categorical_accuracy: 0.6405
Individual Model Double Size in Epoch 14 on Training Set
8675/8675 [==============================] - 2342s 270ms/step - loss: 1.1947 - categorical_accuracy: 0.6504
Individual Model Double Size in Epoch 15 on Training Set
8675/8675 [==============================] - 2337s 269ms/step - loss: 1.2139 - categorical_accuracy: 0.6445
Individual Model Double Size in Epoch 15 on Training Set
8675/8675 [==============================] - 2233s 257ms/step - loss: 1.2460 - categorical_accuracy: 0.6427
Individual Model Double Size in Epoch 16 on Training Set
8675/8675 [==============================] - 2220s 256ms/step - loss: 1.2236 - categorical_accuracy: 0.6512
Individual Model Double Size in Epoch 16 on Training Set
8675/8675 [==============================] - 2238s 258ms/step - loss: 1.2575 - categorical_accuracy: 0.6382
Individual